In [1]:
import os
input_name = "ky2.png"
base_name = os.path.splitext(input_name)[0]

building_map_name = f"{base_name}_building_map.png" #raster of buildings only
input_raster = building_map_name  # binary or probability raster
output_geojson = "f{base_name}.geojson"

In [3]:
import cv2
import numpy as np

# Load image
img = cv2.imread(input_name)  # BGR format in OpenCV

# Split channels
b, g, r = cv2.split(img)

# Define threshold for red detection
threshold = 100  # tweak as needed

# Create binary mask: 1 if red is strong and green/blue are weak
building_map = np.where((r > threshold) & (g < threshold) & (b < threshold), 1, 0)

# Optional: save as grayscale image
cv2.imwrite(building_map_name, (building_map * 255).astype(np.uint8))

True

In [45]:
import rasterio
from rasterio.features import shapes
import numpy as np
from skimage import measure, morphology
from shapely.geometry import shape, mapping
import fiona
import json

input_raster = building_map_name  # binary or probability raster
output_geojson = "ky2_buildings.geojson"
threshold = 0.5                     # if probability raster
simplify_tolerance = 2.0            # units in arbitrary coordinate system
min_area_pixels = 50


import matplotlib.pyplot as plt
def visualise(mask, figsize = (18,18)):
    plt.figure(figsize = figsize)
    plt.imshow(mask, cmap='gray')
    # plt.title("Cleaned Binary Mask")
    plt.axis('off')  # Hide axes for cleaner look
    plt.show()

In [ ]:
# Load raster
with rasterio.open(input_raster) as src:
    raster = src.read(1)


# Threshold if raster is soft probability
mask = raster > threshold

# Remove small objects (noise)
mask = morphology.remove_small_objects(mask.astype(bool), min_size=min_area_pixels)
# visualise(mask)
